This is evaluation for トキポナ1000文コーパス by ponaLM

PonaLM is still weak to evaluate correctly. So the fluency score might be wrong.

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import os, sys, re, regex
sys.path.append("../../../ponaLM")
from models import PonaLM
import nltk
pd.set_option("display.max_colwidth", 1000)

prepare toki pona language model and dictionary

In [2]:
with open('../../words.txt') as f:
    words = [word.rstrip() for word in f.readlines()]
    w2n = {w:i for i,w in enumerate(words)}

# for detect proper noun
def proper(word):
    return re.match(r'^([AIUEO]|[KSNPML][aiueo]|[TJ][aueo]|W[aie])n?(([ksnpml]?[aiueo]|[tj][aueo]|w[aie])n?)*$',word)

# for detect symbols
p_emoji = regex.compile(r'\p{Emoji=Yes}+')

def word_to_id(word):
    if word in w2n:
        return w2n[word]
    elif proper(word):
        return w2n['<Q>']
    elif word.isnumeric():
        return w2n['<NUM>']
    elif p_emoji.search(word):
        return w2n['<SYM>']
    else:
        assert mopu

In [3]:
model = PonaLM(len(words))
model.load_state_dict(torch.load('../../../ponaLM/lm.model', map_location='cpu'))
model.eval()

PonaLM(
  (embed): Embedding(145, 20)
  (lstmf): LSTM(20, 300)
  (lstmb): LSTM(20, 300)
  (h2h): Linear(in_features=300, out_features=20, bias=True)
  (h2o): Linear(in_features=20, out_features=145, bias=True)
  (drop1): Dropout(p=0.03)
  (drop2): Dropout(p=0.03)
  (drop3): Dropout(p=0.2)
  (drop4): Dropout(p=0.2)
)

code for evaluation

In [4]:
# Negotive Log Likelihhod
def NLL(sent):
    sent = [word_to_id(w) for w in sent]
    pred = model(torch.tensor([[0,0] + sent]), torch.tensor([[0,0] + sent[::-1]]))
    return [-p[w].item() for p,w in zip(pred[1:-1], sent)]

# evaluate the score of sentences
def evaluate(sent): 
    nlls = NLL(sent)
    prob = [round(np.exp(-l)*100,2) for l in nlls] # probability = exp(-NLL)
    fluency = round(1/(1+np.mean(nlls)),3)         # fluency = 1 / (1 + ave(all NLL))
    display(pd.DataFrame([prob + [fluency]], index=['prob'], columns=sent + ['Fluency']))

load corpus

In [5]:
sents = [sent.rstrip().split(' ') for sent in open('pona_corpus1000.txt')]
sents[0]

['mi', 'jan', 'Alito', '.']

evaluate fluency of each sentence

In [6]:
for i, line in enumerate(sents):
    print('#{:04}'.format(i+1))
    evaluate(line)

#0001


,mi,jan,Alito,.,Fluency
prob,2.05,45.35,54.87,88.78,0.426


#0002


,ali,li,ike,tawa,ona,.,Fluency
prob,1.2,79.97,6.97,53.46,8.49,90.12,0.363


#0003


...

,tenpo,seme,la,sina,sona,e,pona,jan,?,Fluency
prob,10.9,3.75,80.59,25.22,7.17,24.41,3.21,0.33,10.34,0.285


#0099


,tomo,tawa,pi,esun,unpa,li,tawa,li,pana,e,kalama,musi,nasa,lon,ma,tomo,.,Fluency
prob,1.26,15.67,0.79,0.36,0.11,19.37,1.5,57.05,11.96,96.08,4.04,4.38,0.57,30.32,65.95,1.83,83.79,0.256


#0100


,ni,li,ijo,pi,nanpa,ale,.,Fluency
prob,16.74,50.43,1.58,59.49,0.96,1.1,88.53,0.299


some trial

In [7]:
def tokenize(sent):
    return nltk.tokenize.word_tokenize(sent)

In [8]:
evaluate(tokenize('moku mi li kili palisa.'))
evaluate(tokenize('moku mi li kili pi palisa.'))
evaluate(tokenize('moku mi li palisa kili.'))
evaluate(tokenize('moku mi li palisa pi kili.'))

,moku,mi,li,kili,palisa,.,Fluency
prob,0.05,7.18,8.83,0.02,0.03,84.98,0.169


,moku,mi,li,kili,pi,palisa,.,Fluency
prob,0.05,7.18,8.83,0.21,53.97,0.07,84.98,0.206


,moku,mi,li,palisa,kili,.,Fluency
prob,0.05,7.18,14.17,0.06,0.06,86.5,0.18


,moku,mi,li,palisa,pi,kili,.,Fluency
prob,0.05,7.18,14.17,0.11,63.59,0.17,86.5,0.212


In [9]:
evaluate(tokenize('o olin mi taso.'))
evaluate(tokenize('o olin e mi taso.'))

,o,olin,mi,taso,.,Fluency
prob,9.3,0.17,0.39,0.19,87.19,0.195


,o,olin,e,mi,taso,.,Fluency
prob,9.3,1.36,61.26,0.37,0.19,87.19,0.239


In [10]:
evaluate(tokenize('nimi namako li nimi pu ala.'))
evaluate(tokenize('nimi namako li nimi pi pu ala.'))
evaluate(tokenize('nimi pi namako li nimi pi pu ala.'))

,nimi,namako,li,nimi,pu,ala,.,Fluency
prob,0.28,0.06,2.92,0.17,0.02,3.7,83.65,0.165


,nimi,namako,li,nimi,pi,pu,ala,.,Fluency
prob,0.28,0.06,2.92,0.35,33.52,0.01,3.7,83.65,0.181


,nimi,pi,namako,li,nimi,pi,pu,ala,.,Fluency
prob,1.16,19.03,0.03,2.92,0.35,33.52,0.01,3.7,83.65,0.196
